In [1]:
# IMPLEMENTING ML MODELS FOR AIRBNB DATA TO PREDICT PRICES

In [2]:
#import dependecies

import pandas as pd
import numpy as np

In [3]:
#data cleaning for nyc
pd.set_option('display.max_columns',None)

nyc_df = pd.read_csv('listings-nyc.csv')

nyc_df["room_type"].unique()
p1 = pd.get_dummies(nyc_df["room_type"])
nyc_df["neighbourhood_group_cleansed"].unique()
p2 = pd.get_dummies(nyc_df["neighbourhood_group_cleansed"])
p = pd.concat([p1, p2], axis = 1)
nyc_df = pd.concat([nyc_df, p], axis = 1)
nyc_df["host_response_rate"] = nyc_df["host_response_rate"].apply(lambda x: str(x)[:-1] if str(x) != "nan" else x).astype("float64")
nyc_df["host_acceptance_rate"] = nyc_df["host_acceptance_rate"].apply(lambda x: str(x)[:-1] if str(x) != "nan" else x).astype("float64")
nyc_df["host_is_superhost"] = nyc_df["host_is_superhost"].apply(lambda x: 1 if str(x) == "t" else 0 if str(x) == "f" else x).astype("float64")
nyc_df["total_verifications"] = nyc_df["host_verifications"].apply(lambda x: len(x.replace("[", "").replace("]", "").split(",")))
nyc_df["host_has_profile_pic"] = nyc_df["host_has_profile_pic"].apply(lambda x: 1 if str(x) == "t" else 0 if str(x) == "f" else x).astype("float64")
nyc_df["host_identity_verified"] = nyc_df["host_identity_verified"].apply(lambda x: 1 if str(x) == "t" else 0 if str(x) == "f" else x).astype("float64")
nyc_df["instant_bookable"] = nyc_df["instant_bookable"].apply(lambda x: 1 if str(x) == "t" else 0).astype("float64")
nyc_df["bathrooms_text"].unique()
nyc_df["bathrooms_text"]=nyc_df["bathrooms_text"].str.replace('half-bath','0.5')
nyc_df["bathrooms"]=nyc_df['bathrooms_text'].str.extract('([0-9][,.]*[0-9]*)') 
nyc_df["bathrooms_text"]=nyc_df["bathrooms_text"].str.contains('shared', regex=True)
nyc_df["shared_bathrooms"]=nyc_df["bathrooms_text"].apply(lambda x: 1 if x == True else 0).astype("float64")
nyc_df["total_amenities"] = nyc_df["amenities"].apply(lambda x: len(x.replace("[", "").replace("]", "").split(",")))
nyc_df["price"]=nyc_df["price"].replace({'\$':''}, regex = True)
nyc_df["price"]=pd.to_numeric(nyc_df["price"], errors='coerce')

cols_nyc=['host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 
        'total_verifications', 'total_amenities', 'Entire home/apt',
       'Hotel room', 'Private room', 'Shared room', 'Bronx', 'Brooklyn',
       'Manhattan', 'Queens', 'Staten Island', 'instant_bookable', 
        "host_listings_count", "host_total_listings_count", "accommodates", "bedrooms", "beds","bathrooms","shared_bathrooms", 
 "availability_30", "availability_60", "availability_90", "availability_365",
 "minimum_nights", "number_of_reviews", 'number_of_reviews_ltm',
       'number_of_reviews_l30d',"reviews_per_month", "review_scores_rating"]


nyc_df[cols_nyc]=nyc_df[cols_nyc].fillna(0)
nyc_df["price"]=nyc_df["price"].fillna(0)
df=nyc_df[(nyc_df["price"]< 500) & (nyc_df["price"]!=0)]
df[cols_nyc]


/Users/chellseacarmel/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (67) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,host_response_rate,host_acceptance_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,total_verifications,total_amenities,Entire home/apt,Hotel room,Private room,Shared room,Bronx,Brooklyn,Manhattan,Queens,Staten Island,instant_bookable,host_listings_count,host_total_listings_count,accommodates,bedrooms,beds,bathrooms,shared_bathrooms,availability_30,availability_60,availability_90,availability_365,minimum_nights,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,reviews_per_month,review_scores_rating
0,35.0,18.0,0.0,1.0,1.0,9,31,1,0,0,0,0,0,1,0,0,0.0,8.0,8.0,1,0.0,1.0,1,0.0,0,17,47,322,30,48,0,0,0.33,4.70
1,21.0,76.0,0.0,1.0,1.0,6,35,1,0,0,0,0,1,0,0,0,0.0,1.0,1.0,3,1.0,3.0,1,0.0,4,10,33,220,1,408,38,0,4.91,4.46
2,100.0,100.0,0.0,1.0,1.0,8,5,0,0,1,0,0,1,0,0,0,0.0,1.0,1.0,2,1.0,1.0,0,0.0,30,60,90,365,30,50,0,0,0.53,4.52
3,100.0,25.0,0.0,1.0,1.0,3,14,1,0,0,0,0,1,0,0,0,0.0,1.0,1.0,4,2.0,2.0,1.5,0.0,0,2,3,91,5,2,1,0,0.02,5.00
4,100.0,100.0,0.0,1.0,0.0,4,25,0,0,1,0,0,0,1,0,0,0.0,1.0,1.0,2,1.0,1.0,1,0.0,4,4,26,218,2,505,31,4,3.70,4.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37707,97.0,18.0,0.0,1.0,1.0,7,21,0,0,1,0,0,0,1,0,0,0.0,219.0,219.0,1,1.0,1.0,1,1.0,0,0,1,276,90,0,0,0,0.00,0.00
37708,97.0,18.0,0.0,1.0,1.0,7,20,0,0,1,0,0,1,0,0,0,0.0,219.0,219.0,1,1.0,1.0,2,1.0,29,59,89,364,30,0,0,0,0.00,0.00
37709,97.0,18.0,0.0,1.0,1.0,7,22,0,0,1,0,0,0,1,0,0,0.0,219.0,219.0,1,1.0,0.0,1,1.0,29,59,89,364,30,0,0,0,0.00,0.00
37711,97.0,18.0,0.0,1.0,1.0,7,22,0,0,1,0,0,0,1,0,0,0.0,219.0,219.0,1,1.0,0.0,1,1.0,29,59,89,364,30,0,0,0,0.00,0.00


In [4]:
#data cleaning for la

la_df=pd.read_csv('listings-la.csv')

la_df["room_type"].unique()
p1 = pd.get_dummies(la_df["room_type"])
la_df["neighbourhood_group_cleansed"].unique()
p2 = pd.get_dummies(la_df["neighbourhood_group_cleansed"])
p = pd.concat([p1, p2], axis = 1)
la_df = pd.concat([la_df, p], axis = 1)
la_df["host_response_rate"] = la_df["host_response_rate"].apply(lambda x: str(x)[:-1] if str(x) != "nan" else x).astype("float64")
la_df["host_acceptance_rate"] = la_df["host_acceptance_rate"].apply(lambda x: str(x)[:-1] if str(x) != "nan" else x).astype("float64")
la_df["host_is_superhost"] = la_df["host_is_superhost"].apply(lambda x: 1 if str(x) == "t" else 0 if str(x) == "f" else x).astype("float64")
la_df["total_verifications"] = la_df["host_verifications"].apply(lambda x: len(x.replace("[", "").replace("]", "").split(",")))
la_df["host_has_profile_pic"] = la_df["host_has_profile_pic"].apply(lambda x: 1 if str(x) == "t" else 0 if str(x) == "f" else x).astype("float64")
la_df["host_identity_verified"] = la_df["host_identity_verified"].apply(lambda x: 1 if str(x) == "t" else 0 if str(x) == "f" else x).astype("float64")
la_df["instant_bookable"] = la_df["instant_bookable"].apply(lambda x: 1 if str(x) == "t" else 0).astype("float64")
la_df["bathrooms_text"].unique()
la_df["bathrooms_text"]=la_df["bathrooms_text"].str.replace('half-bath','0.5')
la_df["bathrooms"]=la_df['bathrooms_text'].str.extract('([0-9][,.]*[0-9]*)') 
la_df["bathrooms_text"]=la_df["bathrooms_text"].str.contains('shared', regex=True)
la_df["shared_bathrooms"]=la_df["bathrooms_text"].apply(lambda x: 1 if x == True else 0).astype("float64")
la_df["total_amenities"] = la_df["amenities"].apply(lambda x: len(x.replace("[", "").replace("]", "").split(",")))
la_df["price"]=la_df["price"].replace({'\$':''}, regex = True)
la_df["price"]=pd.to_numeric(la_df["price"], errors='coerce')

cols_la = ['host_is_superhost', 'instant_bookable', 'host_total_listings_count', 
        'total_verifications', 
        'City of Los Angeles',
        'Shared room', 'Private room', 'Hotel room', 'Entire home/apt',
         #'Other Cities', 'Unincorporated Areas',
        'total_amenities', 
        'accommodates','bedrooms', 'beds', 'bathrooms','shared_bathrooms',
        "availability_30", "availability_365", 
        "minimum_nights", "number_of_reviews"]

la_df[cols_la]=la_df[cols_la].fillna(0)
la_df["price"]=la_df["price"].fillna(0)
df2=la_df[(la_df["price"]< 500) & (la_df["price"]!=0)]
df2[cols_la]


/Users/chellseacarmel/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (58,59) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,host_is_superhost,instant_bookable,host_total_listings_count,total_verifications,City of Los Angeles,Shared room,Private room,Hotel room,Entire home/apt,total_amenities,accommodates,bedrooms,beds,bathrooms,shared_bathrooms,availability_30,availability_365,minimum_nights,number_of_reviews
0,0.0,0.0,1.0,8,0,0,0,0,1,25,6,2.0,3.0,2,0.0,0,193,30,2
1,1.0,1.0,2.0,7,1,0,1,0,0,40,1,1.0,1.0,1,1.0,6,300,30,33
2,1.0,0.0,2.0,8,0,0,1,0,0,19,1,1.0,1.0,1,0.0,22,357,7,21
3,0.0,1.0,1.0,4,0,0,0,0,1,23,2,1.0,1.0,1,0.0,0,0,30,0
4,0.0,0.0,6.0,5,1,0,0,0,1,21,2,1.0,2.0,1,0.0,0,274,31,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32871,0.0,1.0,0.0,6,1,0,1,0,0,19,2,0.0,1.0,1,0.0,17,352,30,0
32873,0.0,1.0,437.0,2,1,0,0,0,1,26,3,1.0,0.0,1,0.0,30,365,30,0
32874,1.0,1.0,22.0,8,1,0,0,0,1,50,2,1.0,1.0,1,0.0,30,365,31,0
32875,0.0,1.0,437.0,2,1,0,0,0,1,26,3,1.0,0.0,1,0.0,30,365,30,0


In [5]:
# NYC 
X_nyc=pd.DataFrame(df[cols_nyc]).to_numpy()
y_nyc=pd.DataFrame(df["price"]).to_numpy()
y_nyc=np.log(y_nyc)
print(X_nyc.shape)
print(y_nyc.shape)

nyc_train= 25000

# LA
X_la=pd.DataFrame(df2[cols_la]).to_numpy()
y_la=pd.DataFrame(df2["price"]).to_numpy()
y_la=np.log(y_la)
print(X_la.shape)
print(y_la.shape)

la_train = 20000

(36295, 34)
(36295, 1)
(30205, 19)
(30205, 1)


In [6]:
#LINEAR REGRESSION FOR NYC
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

n_samples, n_features = nyc_train,X_nyc.shape[1]
reg = LinearRegression()  
reg.fit(X_nyc[:nyc_train], y_nyc[:nyc_train])  
y_nyc_pred = reg.predict(X_nyc[nyc_train:])

print("Mean squared error: %.2f" % mean_squared_error(y_nyc[nyc_train:], y_nyc_pred))
print('Coefficients: \n', reg.coef_)
print("R2:", r2_score(y_nyc[nyc_train:,:], y_nyc_pred))

Mean squared error: 0.18
Coefficients: 
 [[-7.74332978e-05 -6.74684013e-04  4.99675501e-02 -3.48474792e-02
   3.40959182e-02 -4.94392802e-03  5.74908675e-03  1.94731959e-01
   4.80089799e-01 -2.69992447e-01 -4.04829310e-01 -1.57911928e-01
   6.86440837e-02  3.45484087e-01 -6.69646098e-02 -1.89251634e-01
   1.52393011e-02  1.94122801e-04  1.94122801e-04  7.86883823e-02
   8.96446574e-02 -1.38666954e-02  6.43073496e-02 -2.00826238e-01
   1.12483605e-02 -6.86828343e-04  6.71576229e-04 -6.70049246e-05
  -1.11657910e-03 -4.01463870e-04 -4.05347404e-04  7.13874846e-03
  -8.16713845e-04 -1.35844000e-02]]
R2: 0.5886459897183789


In [7]:
#LINEAR REGRESSION FOR LA

n_samples, n_features = la_train,X_la.shape[1]
reg = LinearRegression()             
reg.fit(X_la[:la_train], y_la[:la_train])  
y_la_pred = reg.predict(X_la[la_train:])

print("Mean squared error: %.2f" % mean_squared_error(y_la[la_train:], y_la_pred))
print('Coefficients: \n', reg.coef_)
print("R2:", r2_score(y_la[la_train:,:], y_la_pred))

Mean squared error: 0.18
Coefficients: 
 [[ 6.32701856e-02 -9.70272881e-03  1.95548340e-04 -3.30275466e-03
   6.28586942e-03 -5.62484252e-01 -6.90109103e-02  2.75838091e-01
   3.55657071e-01  3.06301873e-03  5.43255133e-02  2.06216345e-01
  -3.03957811e-02  2.56772274e-02 -2.89471465e-01  1.05849201e-02
  -7.53999894e-05 -7.30615628e-04 -1.03949792e-04]]
R2: 0.5896066966048767


In [8]:
# RIDGE REGRESSION FOR NYC

from sklearn.linear_model import Ridge

n_samples, n_features = nyc_train,X_nyc.shape[1]
rdg = Ridge(alpha = 7)  #
rdg.fit(X_nyc[:nyc_train], y_nyc[:nyc_train])  
y_nyc_pred = rdg.predict(X_nyc[nyc_train:])

print("Mean squared error: %.2f" % mean_squared_error(y_nyc[nyc_train:], y_nyc_pred))
print('Coefficients: \n', rdg.coef_)
print("R2:", r2_score(y_nyc[nyc_train:,:], y_nyc_pred))


Mean squared error: 0.18
Coefficients: 
 [[-7.68444692e-05 -6.72387332e-04  4.95886937e-02 -3.16874510e-02
   3.43755658e-02 -5.03708472e-03  5.75070732e-03  1.97722597e-01
   4.62249796e-01 -2.66132244e-01 -3.93840149e-01 -1.57916166e-01
   6.71708109e-02  3.44018605e-01 -6.83758739e-02 -1.84897376e-01
   1.55595460e-02  1.94113193e-04  1.94113188e-04  7.88153067e-02
   8.95000724e-02 -1.39040834e-02  6.39913328e-02 -2.01595987e-01
   1.12598513e-02 -6.82912151e-04  6.63206114e-04 -6.59054643e-05
  -1.11738952e-03 -4.02914756e-04 -3.71988146e-04  6.89098606e-03
  -8.17655272e-04 -1.35642418e-02]]
R2: 0.5887751586403095


In [9]:
#RIDGE REGRESSION FOR LA

n_samples, n_features = la_train,X_la.shape[1]
rdg = Ridge(alpha = 10)             
rdg.fit(X_la[:la_train], y_la[:la_train])  
y_la_pred = rdg.predict(X_la[la_train:])

print("Mean squared error: %.2f" % mean_squared_error(y_la[la_train:], y_la_pred))
print('Coefficients: \n', rdg.coef_)
print("R2:", r2_score(y_la[la_train:,:], y_la_pred))


Mean squared error: 0.18
Coefficients: 
 [[ 6.31213008e-02 -9.29171003e-03  1.95596808e-04 -3.28965256e-03
   6.05737372e-03 -5.46870936e-01 -6.41785389e-02  2.51765423e-01
   3.59284052e-01  3.08161416e-03  5.45508446e-02  2.06062079e-01
  -3.05920275e-02  2.54485161e-02 -2.90747168e-01  1.05783415e-02
  -7.54209325e-05 -7.30693176e-04 -1.02792066e-04]]
R2: 0.5896813534331058


In [10]:
# SVM for NYC
from sklearn.svm import LinearSVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

n_samples, n_features = nyc_train,X_nyc.shape[1]
regr = make_pipeline(StandardScaler(),LinearSVR(random_state=0, tol=1e-5))
regr.fit(X_nyc[:nyc_train], y_nyc[:nyc_train])
y_nyc_pred = regr.predict(X_nyc[nyc_train:])

print("Mean squared error: %.2f" % mean_squared_error(y_nyc[nyc_train:], y_nyc_pred))
print("R2:", r2_score(y_nyc[nyc_train:,:], y_nyc_pred))

/Users/chellseacarmel/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean squared error: 0.18
R2: 0.5940333865300378


/Users/chellseacarmel/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [11]:
#SVM for LA
from sklearn.svm import LinearSVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

n_samples, n_features = la_train,X_la.shape[1]
regr = make_pipeline(StandardScaler(),LinearSVR(random_state=0, tol=1e-5))
regr.fit(X_la[:la_train], y_la[:la_train])
y_la_pred = regr.predict(X_la[la_train:])

print("Mean squared error: %.2f" % mean_squared_error(y_la[la_train:], y_la_pred))
print("R2:", r2_score(y_la[la_train:,:], y_la_pred))


/Users/chellseacarmel/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean squared error: 0.18
R2: 0.5828374719470867


/Users/chellseacarmel/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [12]:
#RANDOM FOREST REGRESSOR FOR NYC
from sklearn.ensemble import RandomForestRegressor

n_samples, n_features = nyc_train,X_nyc.shape[1]
regr = RandomForestRegressor(max_depth=9, random_state=42)
regr.fit(X_nyc[:nyc_train], y_nyc[:nyc_train])
y_nyc_pred = regr.predict(X_nyc[nyc_train:])

print("Mean squared error: %.2f" % mean_squared_error(y_nyc[nyc_train:], y_nyc_pred))
print("R2:", r2_score(y_nyc[nyc_train:,:], y_nyc_pred))

/var/folders/mf/r9n0tb5s5_d0kp6p05vwrfqm0000gn/T/ipykernel_21314/3178122885.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regr.fit(X_nyc[:nyc_train], y_nyc[:nyc_train])


Mean squared error: 0.15
R2: 0.6685350088925464


In [13]:
#RANDOM FOREST REGRESSOR FOR LA

n_samples, n_features = la_train,X_nyc.shape[1]
regr = RandomForestRegressor(max_depth=9, random_state=42)
regr.fit(X_la[:la_train], y_la[:la_train])
y_la_pred = regr.predict(X_la[la_train:])

print("Mean squared error: %.2f" % mean_squared_error(y_la[la_train:], y_la_pred))
print("R2:", r2_score(y_la[la_train:,:], y_la_pred))

/var/folders/mf/r9n0tb5s5_d0kp6p05vwrfqm0000gn/T/ipykernel_21314/1416372220.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regr.fit(X_la[:la_train], y_la[:la_train])


Mean squared error: 0.16
R2: 0.6261477116914161


In [14]:
#SG BOOST FOR NYC
from sklearn import ensemble
params = {
    "n_estimators": 300,
    "max_depth": 3,
    "min_samples_split": 5,
    "learning_rate": 0.08,
    "loss": "squared_error",
}
regr = ensemble.GradientBoostingRegressor(**params)
regr.fit(X_nyc[:nyc_train], y_nyc[:nyc_train])
y_nyc_pred = regr.predict(X_nyc[nyc_train:])

print("Mean squared error: %.2f" % mean_squared_error(y_nyc[nyc_train:], y_nyc_pred))
print("R2:", r2_score(y_nyc[nyc_train:,:], y_nyc_pred))

/Users/chellseacarmel/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean squared error: 0.14
R2: 0.6884339732936311


In [15]:
#SG BOOST FOR LA
params = {
    "n_estimators": 300,
    "max_depth": 3,
    "min_samples_split": 5,
    "learning_rate": 0.08,
    "loss": "squared_error",
}
regr = ensemble.GradientBoostingRegressor(**params)
regr.fit(X_la[:la_train], y_la[:la_train])
y_la_pred = regr.predict(X_la[la_train:])

print("Mean squared error: %.2f" % mean_squared_error(y_la[la_train:], y_la_pred))
print("R2:", r2_score(y_la[la_train:,:], y_la_pred))

/Users/chellseacarmel/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean squared error: 0.15
R2: 0.6605919100558368
